In [68]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\j

[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading p

True

## Reading File and creating DataFrame

In [142]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pickle

In [ ]:
import pandas as pd
wiki_files = pd.read_csv('wiki_sample.csv')
wiki_dataframe = pd.DataFrame(wiki_files)
wiki_dataframe

In [ ]:
sum = 0
for entry in wiki_dataframe['tokenized_content']:
    sum += len(entry)
    

In [ ]:
sum
len(vocab)

In [ ]:
total_words
len(inv_idx)

## Preprocessing and Creating Vocab

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pycountry
lemmatizer = WordNetLemmatizer()
from names_dataset import NameDatasetV1 # v1
names = NameDatasetV1()

stop_words = set(stopwords.words('english'))
# for domain_stop_word in domain_stop_words:
#     stop_words.add(domain_stop_word)

In [ ]:
import pandas as pd
companies_file = pd.read_csv('companies_sorted.csv')
companies_dataframe = pd.DataFrame(companies_file)
companies_dataframe

In [ ]:
companies = set(companies_dataframe['name'])

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
words = set(nltk.corpus.words.words())
lowerCasedWords = map(lambda word: word.lower(), nltk.corpus.words.words())
lowerCasedWords = set(list(lowerCasedWords))
for country in list(pycountry.countries):
    lowerCasedWords.add(country.name.lower())
for company in companies:
    lowerCasedWords.add(str(company).lower())

In [ ]:
# lemmatization, lowercase, remove non alphanumeric, remove non-english, remove numbers and stopword removal
rejected_content = []
def preprocess_vocab(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        token = lemmatizer.lemmatize(token).lower()
        if names.search_first_name(token) or names.search_last_name(token) or ((token in lowerCasedWords) and (token not in stop_words) and (token.isalpha())):
            filtered_content.append(token) 
        else:
            rejected_content.append(token)
        
    return filtered_content

In [ ]:
wiki_dataframe['tokenized_content'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab(row, len(row['title'])), axis=1)

In [ ]:
wiki_dataframe.to_pickle('./wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl')

In [2]:
import pickle5 as pickle
with open("./data/wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl", "rb") as pickle_file:
    wiki_dataframe = pickle.load(pickle_file)
# wiki_dataframe = pd.read_pickle('./wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl')

In [ ]:
wiki_dataframe

## Creating Part of Inverted Index

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1

In [ ]:
wiki_dataframe['most_frequent_term'] = most_freq

In [ ]:
wiki_dataframe

In [ ]:
from collections import OrderedDict
inv_idx_ordered = OrderedDict(sorted(inv_idx.items(), key=lambda item: len(item[1]), reverse=True))

In [ ]:
words = list(inv_idx_ordered.keys())
domain_stop_words = words[0:10000]

## Heaps Law

In [ ]:
heaps_law_dataset[len(heaps_law_dataset)-1]

In [ ]:
import matplotlib.pyplot as plt
import math

def heaps_law(list_to_graph):
        x = list()
        y = list()
        
        for item in list_to_graph:
            x.append(item[0])
            y.append(item[1])

        plt.plot(x, y)
        plt.xlim(1, x[-1])
        plt.ylim(1, y[-1])
        plt.savefig("heaps_law_words_from_nltk_english_corpus.png")

In [ ]:
heaps_law(heaps_law_dataset)

## Zipfs Law

In [ ]:
import matplotlib.pyplot as plt
import math

def zipfs_law(list_to_graph):
        x = list()
        y = list()
        
        for i, word in enumerate(list_to_graph):
            x.append(i+1)
            y.append(len(inv_idx_ordered[word]))
            

        plt.loglog(x, y)
        plt.savefig("zipfs_law.png")

In [ ]:
zipfs_law(inv_idx_ordered)

## Finishing Inverted Index

In [ ]:
for value in tqdm(inv_idx_ordered.items()):
    inv_idx[value[0]] = (Counter(value[1]))

In [ ]:
inv_idx

In [ ]:
file_to_write = open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "wb")
pickle.dump(inv_idx, file_to_write)

In [5]:
inv_idx = pickle.load(open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "rb"))

## Suggesting Queries

In [ ]:
from os import listdir
from os.path import isfile, join
aol_query_log = pd.read_csv('project_1_AOL_query_log/Clean-Data-01.txt', sep="\t")
for file in listdir('project_1_AOL_query_log')[1:]:
    aol_query_log = aol_query_log.append(pd.read_csv('project_1_AOL_query_log/' + str(file), sep="\t"), ignore_index=True)

In [ ]:
# candidate_scores = []
# for candidate in candidates:
#     num_sessions_q_modified = 0
#     for ID in set(aol_query_log[aol_query_log['Query'] == original_query]['AnonID']):
#         candidate_list = list(aol_query_log[aol_query_log['AnonID'] == ID]['Query'])
#         try:
#             index = candidate_list.index(original_query)
#         except: 
#             continue
#         if candidate[1] in candidate_list[index+1:]:
#             num_sessions_q_modified += 1
#         candidate_scorse.append((candidate, num_sessions_q_modified / num_session_containing_q))

In [ ]:
set(aol_query_log[aol_query_log['Query'] == 'vietnam']['AnonID'])


In [ ]:
aol_query_log

In [ ]:
type(aol_query_log['AnonID'][0])

In [162]:
from collections import defaultdict
import itertools

aol_queries = aol_query_log['Query'].values
def identify_candidate_queries(query):
    print("Identifying Candidate Queries...")
    candidate_queries = defaultdict(int)
    potential = list()
    sessions_with_original_query = set(aol_query_log[aol_query_log['Query'] == query]['AnonID'])
    for anon_id in sessions_with_original_query:
        potential_queries = aol_query_log[aol_query_log['AnonID'] == anon_id]['Query']
        if len(potential_queries) > 1:
            potential.append(potential_queries)
    for aol_query in itertools.chain(*potential):
        if len(str(aol_query).split()) > len(query.split()):
            if aol_query.startswith(query):
                candidate_queries[aol_query] += 1
    print("Done!")
    return candidate_queries, len(sessions_with_original_query)
candidate_queries, num_sessions_containing_q = identify_candidate_queries("men")

Identifying Candidate Queries...


84919it [00:00, 1124619.84it/s]

Done!


In [163]:
len(candidate_queries)

104

#### Ranking Candidate Suggestions
##### 𝑆𝑐𝑜𝑟𝑒(𝐶𝑄, 𝑞′) = \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑖𝑠 𝑚𝑜𝑑𝑖𝑓𝑖𝑒𝑑 𝑡𝑜 𝐶𝑄 ÷ \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑎𝑝𝑝𝑒𝑎𝑟𝑠

In [165]:
def rank_candidate_queries(original_query, candidates, num_sessions_containing_q):
    print("Ranking Candidate Queries")
    candidate_scores = []

    for candidate in tqdm(candidates): 
        if num_sessions_containing_q != 0:
            candidate_scores.append((candidate, candidates[candidate] / num_sessions_containing_q))
   
    if len(candidate_scores) < 1:
        candidate_scores.append((original_query, 0))
    print("Done!")
    return sorted(candidate_scores, key=lambda candidate_score: candidate_score[1], reverse=True)
rank_candidate_queries("men", candidate_queries, num_sessions_containing_q)

Ranking Candidate Queries


100%|██████████| 104/104 [00:00<?, ?it/s]


[('men bodies', 0.18072289156626506),
 ("men's health", 0.08433734939759036),
 ('mens penis', 0.07228915662650602),
 ('mental illness', 0.07228915662650602),
 ('menstrual cycle', 0.07228915662650602),
 ('men penis', 0.060240963855421686),
 ('mens wedding rings', 0.060240963855421686),
 ("men's robe", 0.060240963855421686),
 ('men fucking', 0.04819277108433735),
 ('mens health', 0.03614457831325301),
 ('men and penis', 0.03614457831325301),
 ('menstruating women', 0.03614457831325301),
 ('men having sex with animals', 0.03614457831325301),
 ('mens leather bags', 0.03614457831325301),
 ('mens rings', 0.03614457831325301),
 ('meniscus posterior root', 0.03614457831325301),
 ('mens zip up heavy hooded sweats', 0.03614457831325301),
 ('mens gold necklace', 0.03614457831325301),
 ('men cruising men in pa', 0.03614457831325301),
 ('men and women', 0.024096385542168676),
 ('mens nude body', 0.024096385542168676),
 ('mens wacky hairstyles', 0.024096385542168676),
 ("men's leather wallet", 0.024

In [13]:
def find_rank_candidate_queries(query):
    candidate_list, num_sessions_containing_q = identify_candidate_queries(query)
    return rank_candidate_queries(query, candidate_list, num_sessions_containing_q)

In [ ]:
find_rank_candidate_queries("frozen")

## Relevance Ranking

### Identifying Candidate Resources

In [10]:
from itertools import combinations
def identify_candidate_resources(query):
    print("Identifying Candidate Resources...")
    results = set()
    split_query = query.split()
    n = len(split_query)
    candidate_list = list()
    for term in split_query:
        print(".", end="")
        if len(inv_idx[term]) > 0:
            candidate_list.append(set(inv_idx[term].keys()))
    if len(candidate_list) > 0:
        results = set.intersection(*candidate_list)
    if len(results) <= 50:
        for combination in combinations(split_query, n - 1):
            print(".", end="")
            candidate_list = list()
            for term in combination:
                candidate_list.append(inv_idx[term])
            if len(candidate_list) > 0:
                results = set.intersection(*candidate_list)
            if len(results) > 50:
                break
            else:
                n -= 1
    return results


In [7]:
identify_candidate_resources('sex')

{229377,
 327681,
 278529,
 507907,
 655365,
 737284,
 131080,
 720904,
 737288,
 442384,
 131095,
 229399,
 131097,
 901151,
 1179680,
 1277987,
 524325,
 426023,
 360490,
 1163306,
 458796,
 1163309,
 1163310,
 639024,
 589873,
 524338,
 442417,
 639026,
 426043,
 1163325,
 442430,
 999486,
 458817,
 1097795,
 1032262,
 1130568,
 81993,
 557130,
 655436,
 196685,
 196687,
 229456,
 655442,
 1146962,
 114770,
 278611,
 639059,
 770135,
 1130584,
 1081434,
 245851,
 32860,
 1163358,
 852065,
 1130594,
 737379,
 671846,
 639083,
 573548,
 933995,
 933998,
 442482,
 737394,
 98420,
 999539,
 1065078,
 999543,
 1032311,
 491642,
 163963,
 721020,
 131197,
 32894,
 999547,
 999549,
 999550,
 999551,
 98435,
 1097857,
 180356,
 1048710,
 196743,
 999562,
 163979,
 1032335,
 180369,
 49301,
 704664,
 213145,
 835741,
 1147039,
 1147042,
 508066,
 737443,
 65705,
 327849,
 950443,
 180394,
 966826,
 737453,
 508082,
 1065141,
 245942,
 295095,
 1097911,
 901307,
 786620,
 442557,
 295103,
 57

### TF-IDF
#### 𝑇𝐹(𝑤, 𝑑) = 𝑓𝑟𝑒𝑞(𝑤, 𝑑) ÷ (𝑚𝑎𝑥_𝑑)
#### 𝐼𝐷𝐹(𝑤) = 𝑙𝑜𝑔__2 (𝑁 ÷ 𝑛_𝑤)

In [ ]:
len(wiki_dataframe)

In [12]:
import math
def tf_idf(split_query, document_id):
    score = 0.001 # not zero for normalization

    if document_id == 0:
        print("error in tf-idf function")
        return

    for term in split_query:
        if term not in inv_idx:
            continue
        score += (inv_idx[term][document_id] / wiki_dataframe['most_frequent_term'][document_id - 1][0][1]) * math.log((len(wiki_dataframe) / len(inv_idx[term])), 2)
    return score

### Ranking candidate resources

In [8]:
def rank_candidate_resources(query, candidate_resources):
  ranked_candidates = {}
  for document_id in candidate_resources:
    ranked_candidates[document_id] = tf_idf(query.split(), document_id)
  return sorted(ranked_candidates.items(), key=lambda item: item[1], reverse=True)

In [13]:
def find_and_rank_candidate_resources(query):
    candidates = identify_candidate_resources(query)
    return rank_candidate_resources(query, candidates)
relevant_resources = find_and_rank_candidate_resources("sex")[0]
relevant_resources

(721020, 7.318117658755251)

### Snippet Generation

In [15]:
def get_snippet(query, document_id):
    row = wiki_dataframe[wiki_dataframe['id'] == document_id + 1]
    snippet = (  # tuple in the form (title, sentences)
        row["title"].to_string(index=False),
        generate_sentence_snippets(query, document_id, int(row["title"].str.len())))
    return snippet

In [21]:
wiki_dataframe["title"][0]

'Morocco–Saudi Arabia relations'

In [21]:
import re
def generate_sentence_snippets(query, document_id, len_title):
    pattern = '(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'
    top_two = []  # format is (sentence, cosine_similarity_score)
    vectorized_query = vectorize(query, document_id)
    for sentence in re.split(pattern, wiki_dataframe["content"][document_id][len_title:]):
        sentence = sentence.replace("\r", "").replace("\n", "")  # 4 is for removal of \r\n\r\n for each sentence.
        lsentence = sentence.lower()

        sentence_score = cosine_similarity(vectorized_query, vectorize(lsentence, document_id))
        if len(top_two) < 2:
            top_two.append((sentence, sentence_score))
            top_two.sort(key=lambda item: item[1], reverse=True)
            continue

        for index, entry in enumerate(top_two):  # this loop should only run twice
            if sentence_score > entry[1]:
                top_two[index] = (sentence, sentence_score)
                top_two.sort(key=lambda item: item[1], reverse=True)
                break
    if len(top_two) < 2:
        result = top_two[0][0]
    else:
        result = (top_two[0][0], top_two[1][0])
    return result

In [18]:
def vectorize(phrase, document_id):
    arr = []
    for word in phrase.split():
        arr.append(tf_idf([word], document_id))
    return arr

In [19]:
from scipy import spatial
def cosine_similarity(vectorized_query, vectorized_sentence):
    lenf = max(len(vectorized_query), len(vectorized_sentence))
    for i in range (lenf + 1):
        if len(vectorized_query) == len(vectorized_sentence):
            break
        if len(vectorized_query) < i:
            vectorized_query.append(0.001)
        if len(vectorized_sentence) < i:
            vectorized_sentence.append(0.001)
    return 1 - spatial.distance.cosine(list(vectorized_sentence), list(vectorized_query))

### Putting Everything together

In [9]:
import pickle5 as pickle

def load_files():
    with open("./data/wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl", "rb") as wiki_df:
        wiki_dataframe = pickle.load(wiki_df)
    inv_idx = pickle.load(open("./data/inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "rb"))
    return wiki_dataframe, inv_idx
wiki_dataframe, inv_idx = load_files()

In [16]:
def search(query):
    # print("Generating results...")
    # ranked_candidate_resources = find_and_rank_candidate_resources(query)[0:10]
    results = {}
    # for resource in ranked_candidate_resources:
    #     print("resource", resource, file=sys.stderr)
    #     title, sentences = get_snippet(query, resource[0])
    #     results[resource[0]] = [title, sentences]
    ranked_candidate_queries = find_rank_candidate_queries(query)
    results["query_suggestions"] = []
    for query_suggestion in ranked_candidate_queries[:5]:
        results["query_suggestions"].append(query_suggestion)

    return results

search_results = search("woman")
print("results:")
print(search_results)

Identifying Candidate Queries...
aol query                                    amazon
                    academy award nominee
                     family tree template
                free family tree template
                        free family chart
               free blank geneology chart
                                     file
                           metro district
                     denver county police
                           metro district
                    nike woman sportswear
                                     nike
                                    woman
                    woman track suit nike
                                    woman
                      woman athletic wear
                               gart sport
                     nike fitness apparel
                  steamboat spring resort
                 parking steamboat spring
                                   goggle
                         galapagos iguana
                         galapago

TypeError: unhashable type: 'DataFrame'

In [7]:
# aol_query_log = pickle.load(open("./data/aol_query_log_data.pkl", "rb"))

aol_query_log.iloc[1065383]

AnonID                         3352559
Query                        actor ken
QueryTime          2006-03-01 17:42:50
Processed Query              actor ken
Name: 1065383, dtype: object